In [1]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [3]:
#cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
cache_dir = None#"/home/tx/workspace/cache" 
model_name = "bigscience/bloom-3b" # bigscience/bloom-3b
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
original_model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True, cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

peft_parameters_path = "./ai_detector_peft/"
#original_model.

In [9]:
from peft import PeftModel
model = PeftModel.from_pretrained(original_model,model_id="./ai_detector_peft/")

In [10]:
import torch 
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean',ignore_index=tokenizer.pad_token_id)
def generate(prompt,model_,tokenizer_):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(model_.device)
    #model_.disable_adapter()
    # 生成文本
    inputs = {"input_ids":input_ids, "max_length":512, 
              "num_beams":5, "no_repeat_ngram_size":2,
             "top_k":50, "top_p":0.95}
    with torch.no_grad():
        output = model_.generate(**inputs)
    # 将生成的token解码成文本
    generated_text = tokenizer_.decode(output[0], skip_special_tokens=True)
    return generated_text



In [11]:
def generate_prompt(prompt,feature_text):
    """
    
    """
    text=  f"""
        read the following text, understand its style and words choices:
        {feature_text}.
        is this text generated by AI?
    """

    return text

In [12]:
model = model.cuda()
final_text = generate_prompt("","this is not what I want")
generate(final_text,model_=model,tokenizer_=tokenizer)


/home/tx/.conda/envs/llava/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:368: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'\n        read the following text, understand its style and words choices:\n        this is not what I want.\n        is this text generated by AI?\n    no, the text is written by students'